In [1]:
# Setup
import urllib3
import certifi
from bs4 import BeautifulSoup
import pandas as pd
import time

http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED', ca_certs=certifi.where())

In [2]:
hitters_url = "https://www.numberfire.com/mlb/fantasy/weekly-projections/batters"
pitchers_url = "https://www.numberfire.com/mlb/fantasy/weekly-projections/pitchers"

urls = {
    'H': hitters_url,
    'P': pitchers_url
}

In [5]:
for key, url in urls.items():
    response = http.request("GET", url)
        
    soup = BeautifulSoup(response.data, 'html.parser')
    player_table = soup.find('table', class_='projection-table--fixed')
    stats_table = soup.find('table', class_="no-fix")
    header = [th.text.strip() for th in stats_table.find('thead').select('th')]
    header.insert(0, "Player")
    
    if key == 'H':
        header[header.index('H2B')] = "2B"
        header[header.index('H3B')] = "3B"
    
    full_stats = []
    for row in stats_table.find('tbody').find_all('tr'):
        stats = [td.text.strip() for td in row.find_all('td')]
        stats.insert(0, player_table.find('tr', attrs={'data-row-index': row['data-row-index']}).td.find('span', class_='full').text.strip())
        full_stats.append(stats)
    cols = zip(*full_stats)
    tbl_d = {name: col for name, col in zip(header, cols)}
    df = pd.DataFrame(tbl_d, columns=header)
    df.to_csv('../data/Numberfile_' + time.strftime("%Y_%b_%d_") + key + ".csv")
